In [ ]:
import pandas as pd
import numpy as np
import csv
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Main program loop
running = True
while running:
    print("\nWelcome to AI Personal Assistant")
    print("===============================")
    print("1. Sleep Quality Prediction")
    print("2. Nutrition Recommendation")
    print("3. Symptom to Disease Prediction")
    print("4. Exit")

    model_choice = input("\nSelect a model (1-4): ")

    # Sleep Model
    if model_choice == "1":
        print("\nRunning Sleep Quality Prediction Model...")
        data = pd.read_csv(r"Sleep_health_and_lifestyle_dataset.csv")
        for col in ['Sleep Duration', 'Heart Rate', 'Physical Activity Level', 'Quality of Sleep']:
            data[col] = data[col].fillna(0)
        x = data[['Sleep Duration', 'Heart Rate', 'Physical Activity Level']]
        y = (data['Quality of Sleep'] >= 6).astype(int)  # Good sleep if quality >=6
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        model = LogisticRegression(max_iter=1000)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model accuracy: {accuracy:.2f} ({accuracy*100:.1f}%)")
        coefficients = pd.DataFrame({
            'Feature': x.columns,
            'Coefficient': model.coef_[0]
        })
        print("\nFeature importance:")
        print(coefficients.sort_values('Coefficient', ascending=False))
        try:
            print("\nPredict your sleep quality:")
            d = float(input("Enter sleep duration (hours): "))
            h = int(input("Enter average heart rate: "))
            a = int(input("Enter physical activity level (minutes): "))
            input_df = pd.DataFrame([[d, h, a]], columns=x.columns)
            
            p = model.predict(input_df)
            prob = model.predict_proba(input_df)[0][1]
            
            print(f"\nPrediction: {'Good Sleep Quality' if p[0] == 1 else 'Poor Sleep Quality'}")
            print(f"Confidence: {prob:.2f} ({prob*100:.1f}%)")
            
            if p[0] == 1:
                print("\nFactors that contributed to good sleep prediction:")
            else:
                print("\nSuggestions to improve sleep quality:")   
            
            if d < 7:
                print(f"- Sleep duration ({d} hours) could be improved. 7-9 hours is recommended.")
            if a < 30:
                print(f"- Physical activity level ({a} minutes) is low. Try to get at least 30 minutes daily.")
            if h > 80:
                print(f"- Heart rate ({h} bpm) is elevated. Relaxation techniques might help.")    
        except ValueError:
            print("Please enter valid numbers for all inputs.")
        print("Thank you for using Sleep and Physical Activity Recommendation System!")

    # Nutrition Model
    elif model_choice == "2":
        print("\nRunning Nutrition Recommendation Model...")
        file_path = (r"nutrition.csv")
        df = pd.read_csv(file_path)
        df = df.dropna(subset=['calories', 'carbohydrate', 'protein', 'fat'], how='all')
        features = df[['calories', 'carbohydrate', 'protein', 'fat']].copy()
        features.columns = ['Calories', 'Carbohydrate (g)', 'Protein (g)', 'Total lipid (fat) (g)']
        for col in features.columns:
            features[col] = features[col].astype(str).str.replace(r'[^\d\.]', '', regex=True)
            features[col] = pd.to_numeric(features[col], errors='coerce')
        features = features.dropna()
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(features)
        knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
        knn.fit(scaled_features)
        print("\nNutrition Recommendation System")
        print("==============================")
        print("\nOptions:")
        print("1. Find foods similar to specific nutritional values")
        print("2. Find low-carb foods")
        print("3. Find high-protein foods")
        choice = input("\nEnter your choice (1-3): ")
        # Process selected option
        if choice == '1':
            calories = float(input("Enter desired calories: "))
            carbs = float(input("Enter desired carbohydrates (g): "))
            protein = float(input("Enter desired protein (g): "))
            fat = float(input("Enter desired fat (g): "))
            query = [calories, carbs, protein, fat]
            
        elif choice == '2':
            query = [300, 5, 20, 10]  
            print("\nFinding Low-Carb Food Recommendations:")
            
        elif choice == '3':
            query = [300, 15, 30, 5]  
            print("\nFinding High-Protein Food Recommendations:")
            
        else:
            print("Invalid choice.")
            continue
        query_df = pd.DataFrame([query], columns=features.columns)   
        scaled_query = scaler.transform(query_df)
        distances, indices = knn.kneighbors(scaled_query, n_neighbors=5)
        food_names = df['name'] if 'name' in df.columns else df.iloc[:, 0]
        print("\nTop 5 Food Recommendations:\n")
        for i, idx in enumerate(indices[0]):
            name = food_names.iloc[idx]
            calories = features.iloc[idx]['Calories']
            carbs = features.iloc[idx]['Carbohydrate (g)']
            protein = features.iloc[idx]['Protein (g)']
            fat = features.iloc[idx]['Total lipid (fat) (g)']
            
            print(f"{i+1}. {name} - Calories: {calories:.1f}, Carbs: {carbs:.1f}g, Protein: {protein:.1f}g, Fat: {fat:.1f}g")
        print("Thank you for using the Nutrition Recommendation System!")

    # Symptom to Disease Model
    elif model_choice == "3":
        print("\nRunning Symptom to Disease Prediction Model...")
        df = pd.read_csv(r"Disease_symptom_and_patient_profile_dataset.csv")
        symptoms = ['Fever', 'Cough', 'Fatigue', 'Difficulty Breathing']
        for symptom in symptoms:
            df[symptom] = df[symptom].map({'Yes': 1, 'No': 0})
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        df['Disease_Label'] = le.fit_transform(df['Disease'])
        X = df[symptoms]
        y = df['Disease_Label']
        from sklearn.tree import DecisionTreeClassifier
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        dt_model = DecisionTreeClassifier(max_depth=3)
        dt_model.fit(X_train, y_train)
        print("\nDisease Prediction System (Decision Tree)")
        print("========================================")
        user_input = []
        for symptom in symptoms:
            answer = input(f"Do you have {symptom}? (Yes/No): ").strip().lower()
            user_input.append(1 if answer == 'yes' else 0)

        input_data = pd.DataFrame([user_input], columns=symptoms)
        prediction = dt_model.predict(input_data)[0]
        confidence = dt_model.predict_proba(input_data)[0].max()
        
        disease_name = le.inverse_transform([prediction])[0]
        print(f"\nPredicted Disease: {disease_name}")
        print(f"Confidence: {confidence:.2f} ({confidence*100:.1f}%)")
        print("Thank you for using the Disease Prediction System!")

    # Exit option
    elif model_choice == "4":
        print("Thank you for using AI Personal Assistant. Goodbye!")
        running = False

    else:
        print("Invalid choice. Please select a number from 1 to 4.")


Welcome to AI Personal Assistant
1. Sleep Quality Prediction
2. Nutrition Recommendation
3. Symptom to Disease Prediction
4. Exit



Select a model (1-4):  1



Running Sleep Quality Prediction Model...
Model accuracy: 1.00 (100.0%)

Feature importance:
                   Feature  Coefficient
2  Physical Activity Level     0.323656
1               Heart Rate    -0.750677
0           Sleep Duration    -0.784157

Predict your sleep quality:


Enter sleep duration (hours):  5
Enter average heart rate:  30
Enter physical activity level (minutes):  60



Prediction: Good Sleep Quality
Confidence: 1.00 (100.0%)

Factors that contributed to good sleep prediction:
- Sleep duration (5.0 hours) could be improved. 7-9 hours is recommended.
Thank you for using Sleep and Physical Activity Recommendation System!

Welcome to AI Personal Assistant
1. Sleep Quality Prediction
2. Nutrition Recommendation
3. Symptom to Disease Prediction
4. Exit


...


Running Sleep Quality Prediction Model...
Model accuracy: 1.00 (100.0%)

Feature importance:
                   Feature  Coefficient
2  Physical Activity Level     0.323656
1               Heart Rate    -0.750677
0           Sleep Duration    -0.784157

Predict your sleep quality:


Enter sleep duration (hours):  7
Enter average heart rate:  80
Enter physical activity level (minutes):  50



Prediction: Good Sleep Quality
Confidence: 0.90 (89.6%)

Factors that contributed to good sleep prediction:
Thank you for using Sleep and Physical Activity Recommendation System!

Welcome to AI Personal Assistant
1. Sleep Quality Prediction
2. Nutrition Recommendation
3. Symptom to Disease Prediction
4. Exit



Select a model (1-4):  6


Invalid choice. Please select a number from 1 to 4.

Welcome to AI Personal Assistant
1. Sleep Quality Prediction
2. Nutrition Recommendation
3. Symptom to Disease Prediction
4. Exit
